In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import optuna 
import xgboost as xgb
from sklearn.cluster import KMeans
from sklearn.model_selection import GroupKFold
from sklearn.metrics import r2_score
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/uhi-index-data/Training_data_uhi_index_UHI2025-v2.csv')
target = train_df['UHI Index']

In [ ]:
# n_clusters = 50
# coords = train_df[['Latitude', 'Longitude']].values
# kmeans = KMeans(n_clusters=n_clusters, random_state=42)
# train_df['group'] = kmeans.fit_predict(coords)

In [ ]:
from sklearn.preprocessing import StandardScaler
X = pd.read_csv('/kaggle/input/ey-final-boss/ey_train_niche_ds_0.982.csv')
y = train_df['UHI Index']  # Ensure train_df is defined and contains 'UHI Index'
cols = X.columns
X.fillna(0, inplace=True)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = pd.DataFrame(X_scaled, columns=[cols])

# optuna

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import optuna
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

def objective(trial):
    # Intense parameter grid for ExtraTreesRegressor
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 5000),
        'max_depth': trial.suggest_int('max_depth', 10, 100),  # depth can go quite deep
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 50),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 30),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None, 0.3, 0.5, 0.7]),
        'bootstrap': trial.suggest_categorical('bootstrap', [False]),
        'criterion': trial.suggest_categorical('criterion', ['squared_error']),
    }
    
    # If bootstrap is True, consider tuning max_samples as well
    if params['bootstrap']:
        params['max_samples'] = trial.suggest_uniform('max_samples', 0.5, 1.0)
    
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.24, random_state=42)
    
    model = ExtraTreesRegressor(**params, n_jobs=-1, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    score = r2_score(y_val, preds)
    return score

# Create and run the study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=800)

print('Best parameters:', study.best_params)
print('Best CV R2 score:', study.best_value)


In [ ]:
# import optuna
# import numpy as np
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score

# def objective(trial):
#     # Define parameter ranges around your current best values
#     params = {
#         'objective': 'reg:squarederror',
#         'booster': 'gbtree',
#         'tree_method': 'gpu_hist',  # Use "hist" for speed; change to "gpu_hist" if you have GPU support
#         'n_estimators': trial.suggest_int('n_estimators', 2000, 6500),  # around 4200
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.09),  # around 0.035
#         'max_depth': trial.suggest_int('max_depth', 7, 30),  # around 17
#         'min_child_weight': trial.suggest_int('min_child_weight', 3, 25),  # around 15
#         'subsample': trial.suggest_uniform('subsample', 0.65, 1.0),  # around 0.9
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.65, 1.0),  # around 0.9
#         'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.65, 0.9),  # around 0.8
#  #       'gamma': trial.suggest_loguniform('gamma', 1e-9, 5.0),  # minimum loss reduction required to make a split
#         'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-9, 1.0),  # L1 regularization
#         'reg_lambda': trial.suggest_loguniform('reg_lambda', 1.9, 2.0),  # L2 regularization
#         'n_jobs': -1,
#         'enable_categorical': False,
#     }

#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.24, random_state=42)
    
#     model = xgb.XGBRegressor(**params, random_state=42, verbosity=0)
#     model.fit(X_train, y_train)
#     preds = model.predict(X_val)
#     score = r2_score(y_val, preds)
#     return score

# # Create and run the study
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=400)

# print('Best parameters :', study.best_params)
# print('Best CV R2 score :', study.best_value)


In [ ]:
# import optuna
# import numpy as np
# import lightgbm as lgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score

# def objective(trial):
#     params = {
#         'objective': 'regression',
#         'metric': 'rmse',               # metric for training; we'll compute R2 later
#         'boosting_type': 'gbdt',
#         'device': 'gpu',                # Use GPU
#         'gpu_platform_id': 0,           # Default GPU platform id (adjust if needed)
#         'gpu_device_id': 0,             # Default GPU device id (adjust if needed)
#         'n_estimators': trial.suggest_int('n_estimators', 2000, 6500),  # around 4200
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.09),  # around 0.035
#         'max_depth': trial.suggest_int('max_depth', 7, 25),   # around 17
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 25),  # similar to min_child_weight=15
#         'subsample': trial.suggest_uniform('subsample', 0.65, 1.0),           # around 0.9
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.65, 1.0),  # around 0.9
#         'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.65, 1.0), # around 0.8
#         'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),         # L1 regularization
#         'reg_lambda': trial.suggest_loguniform('reg_lambda', 1.0, 2.0),          # L2 regularization
#         'n_jobs': -1,
#         'verbose': -1
#     }
    
#     # Create a holdout split: 80% training, 20% validation.
#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    
#     train_data = lgb.Dataset(X_train, label=y_train)
#     val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)
    
#     model = lgb.train(params, train_data, valid_sets=[val_data])
    
#     preds = model.predict(X_val)
#     score = r2_score(y_val, preds)
#     return score

# # Create an Optuna study (direction is 'maximize' because higher R2 is better)
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=400)

# print('Best parameters :', study.best_params)
# print('Best CV R2 score :', study.best_value)
